## Log-playback (expert) trajectory extraction and usage

This notebook demonstrates how to extract expert actions using different dynamics models and step through scene with those actions. 

Details on the dynamics models are found in [the docs](https://github.com/Emerge-Lab/gpudrive/tree/main/pygpudrive/env).

In [10]:
import os
from pathlib import Path
import mediapy

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)

from pygpudrive.env.config import EnvConfig, RenderConfig, SceneConfig
from pygpudrive.env.env_torch import GPUDriveTorchEnv

### Configurations

In [11]:
DYNAMICS_MODEL = "state" # "delta_local" / "state" / "classic"
DATA_PATH = "data/processed/examples" # Your data path
MAX_NUM_OBJECTS = 128
NUM_ENVS = 4
testing = False 
testing = True

# Configs
render_config = RenderConfig(draw_obj_idx=True)
scene_config = SceneConfig(path=DATA_PATH, num_scenes=NUM_ENVS)
env_config = EnvConfig(dynamics_model=DYNAMICS_MODEL)

### Make environment

In [12]:
env = GPUDriveTorchEnv(
    config=env_config,
    scene_config=scene_config,
    max_cont_agents=MAX_NUM_OBJECTS,
    device="cpu",
    render_config=render_config,
    action_type="discrete" # "continuous" or "discrete"
)


--- Ratio unique scenes / number of worls = 3 / 4 ---



### Get log-playback (expert) actions

- Different dynamics models have different action spaces. For details, [see the docs.](https://github.com/Emerge-Lab/gpudrive/tree/main/pygpudrive/env)

In [13]:
obs = env.reset()

In [14]:
# Extract full expert trajectory
expert_actions, _, _, _ = env.get_expert_actions()

expert_actions.shape # Shape: (num_envs, num_steps, num_agents, num_actions)

torch.Size([4, 32, 91, 10])

### Step through an episode

In [15]:
if (not testing):
    # Reset environment
    obs = env.reset()

    frames = {f"env_{i}": [] for i in range(NUM_ENVS)}

    # Step through the scene
    for t in range(env_config.episode_len):

        env.step_dynamics(expert_actions[:, :, t, :])
        
        # Render the scenes
        for i in range(NUM_ENVS):
            frames[f"env_{i}"].append(env.render(i))



In [16]:
if (testing):

    import matplotlib.pyplot as plt

    # Reset environment
    obs = env.reset()

    frames = {f"env_{i}": [] for i in range(NUM_ENVS)}

    # Step through the scene
    for t in range(env_config.episode_len):

        env.step_dynamics(expert_actions[:, :, t, :])
        
        # Render the scenes
        # NUM_ENVS = 2 # NOTE: for debugging
        for i in range(1):
            frames[f"env_{i}"].append(env.render(i))

    plt.imshow(frames["env_0"][4])
    plt.axis("off")
    plt.show()


In [17]:
# plt.imshow(frames["env_0"][4])
# plt.axis("off")
# plt.show()

### Show resulting trajectories!

In [18]:
if (not testing):
    # Show videos
    mediapy.show_videos(frames, fps=30, width=600, height=400, columns=2)

env_0 This browser does not support the video tag.,env_1 This browser does not support the video tag.
env_2 This browser does not support the video tag.,env_3 This browser does not support the video tag.
